In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the pre-trained model and tokenizer
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Load the CSV file with the correct encoding
csv_path = "/content/d1.csv"
df = pd.read_csv(csv_path, encoding='ISO-8859-1')

question = input()

def is_age_question(question):
    return "youngest" in question.lower() or "oldest" in question.lower()

def calculate_age(date_of_birth, current_year):
    if date_of_birth is None:
        return None
    birth_year = int(date_of_birth.split("-")[0])
    return current_year - birth_year

# Iterate through the contexts and find the best answer
best_answer = None
best_score = float('-inf')

for row in df.itertuples(index=False):
    context = row.Data

    # Tokenize the input
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")

    # Perform the question answering
    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Get the most probable answer
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

    # Keep track of the best answer
    score = start_scores[0][answer_start] + end_scores[0][answer_end - 1]
    if score > best_score:
        best_score = score
        best_answer = answer

# Check if the question is about "youngest" or "oldest" player
if is_age_question(question):
    current_year = 2023  # Replace with the actual current year
    target_attribute = "date of birth"  # Adjust based on your dataset
    target_player = None
    target_value = float('inf') if "youngest" in question.lower() else 0

    for row in df.itertuples(index=False):
        context = row.Data
        row_dict = row._asdict()
        dob = row_dict.get(target_attribute)  # Access attribute by name using dictionary indexing
        age = calculate_age(dob, current_year)

        # Handle None values by skipping the comparison
        if age is None:
            continue

        if "youngest" in question.lower() and age < target_value:
            target_value = age
            target_player = context
        elif "oldest" in question.lower() and age > target_value:
            target_value = age
            target_player = context

    best_answer = target_player

print("Question:", question)
print("Answer:", best_answer)



How height is Suso?
Question: How height is Suso?
Answer:  177 meters
